<a href="https://colab.research.google.com/github/randiepathirage/4th/blob/main/implementation/Imp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import re
import matplotlib.pyplot as plt
from tensorflow import keras
from unicodedata import normalize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import LSTM,Bidirectional,Input,Concatenate
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# DATA_PATH = '/content/drive/MyDrive/UNI/4th Year/Research/DataSet/LTRL_Clean.tsv'
DATA_PATH = '/content/drive/MyDrive/4th year Research/data collection/data/datasets/1200_dataset.csv'

model_id = "e5_1"
batch_size = 32  # Batch size for training.
epochs = 10 # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1200 # Number of samples to train on.
num_test = 200

df = pd.read_csv(DATA_PATH, names=["singlish","sinhala"],dtype=str).astype(str)
df = df.sample(frac=1).reset_index(drop=True)

input_data = pd.DataFrame(df.singlish.values[:num_samples])
input_data.columns = ['singlish']

target_data = pd.DataFrame(df.sinhala.values[:num_samples])
target_data.columns = ['sinhala']

In [ ]:
len(target_data)

1200

In [ ]:
input_data

,singlish
0,poliyat
1,hmmeoyala
2,simawak
3,onene
4,uganthaya
...,...
1195,kemathiy
1196,oyalatayh
1197,mawamai
1198,gnnathuw


In [ ]:
def normalize_text(text):
  text = normalize('NFD', text).encode('utf-8')
  return text.decode('UTF-8')

def clean_and_normalize(text):
  text = normalize_text(text.lower())
  return text

def removeUnnecessaryRepeats(word):
  prev = word[0]
  repeatCount = 1
  ans = word[0]
  for i in range(1,len(word)):
    if(word[i]==prev):
      repeatCount+=1
    else:
      repeatCount=1
    
    if(repeatCount<3):
      ans+=word[i]
      prev = word[i]

  return ans

def preProcessSinglish(singText):
  vowels =  ['a','e','i','o','u']
  specialConsonentSuffix = ['h']
  singText = singText.lower()
  singWords = singText.split()
  finList = []
  for singWord in singWords:
    singWord = removeUnnecessaryRepeats(singWord)
    buf = ""
    for i in range(len(singWord)):
      char = singWord[i]

      if(char in vowels):
        if(i<len(singWord)-1):
          if(char == singWord[i+1]):
            buf+=char
          else:
            buf+=char
            finList.append(buf)
            buf=""
        else:
          buf+=char
          finList.append(buf)
          buf=""
      else:
        if(i<len(singWord)-1):
          if(char == singWord[i+1] or (singWord[i+1] not in vowels and singWord[i+1] not in specialConsonentSuffix)):
            buf+=char
            finList.append(buf)
            buf=""
          else:
            buf+=char
        else:
          buf+=char
          finList.append(buf)
          buf=""
    finList.append("$")

  return " ".join(finList)

def preProcessSinhala(sinText):
  sinWords = sinText.split()
  finList = []
  for sinWord in sinWords:
    charList = re.findall(r'\w\W?',sinWord.strip())
    finList.extend(charList)
    finList.append("$")
  return " ".join(finList)

input_data['singlish'] = input_data['singlish'].apply(lambda x: preProcessSinglish(clean_and_normalize(x)))
target_data['sinhala'] = target_data['sinhala'].apply(lambda x: preProcessSinhala(clean_and_normalize(x))).apply(lambda x: '<S> '+x+' <E>')


In [ ]:
target_data

,sinhala
0,<S> පෙ ලි ය ත් $ <E>
1,<S> හ් ම් මෙ ඔ යා ල $ <E>
2,<S> සී මා ව ක් $ <E>
3,<S> ඕ න නෙ $ <E>
4,<S> වෙ න $ <E>
...,...
1195,<S> කැ ම ති යි $ <E>
1196,<S> ඔ යා ල ට ත් $ <E>
1197,<S> මා ව ම යි $ <E>
1198,<S> ග න් නැ තු ව $ <E>


In [ ]:
max_encoder_length = 250
max_decoder_length = 250

# Vectorize the data.
input_texts = []
target_texts = []
input_TUs = set()
target_TUs = set()

input_texts = input_data.singlish
target_texts = target_data.sinhala

for input_text in input_texts:
  for TU in input_text.split():
      if TU not in input_TUs:
          input_TUs.add(TU)
  
for target_text in target_texts:
  for TU in target_text.split():
      if TU not in target_TUs:
          target_TUs.add(TU)

input_TUs = sorted(list(input_TUs))
target_TUs = sorted(list(target_TUs))
num_encoder_tokens = len(input_TUs)
num_decoder_tokens = len(target_TUs)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(TU, i) for i, TU in enumerate(input_TUs)])
target_token_index = dict([(TU, i) for i, TU in enumerate(target_TUs)])

Number of samples: 1200
Number of unique input tokens: 179
Number of unique output tokens: 209
Max sequence length for inputs: 30
Max sequence length for outputs: 33


In [ ]:
target_token_index

In [ ]:
input_token_index

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, TU in enumerate(input_text.split()):
        encoder_input_data[i, t, input_token_index[TU]] = 1.0
    for t, TU in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[TU]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[TU]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index["$"]] = 1.0
    decoder_target_data[i, t:, target_token_index["$"]] = 1.0

target_texts[0].split()

encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.Bidirectional(keras.layers.LSTM(latent_dim, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

print(state_h.shape)

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(latent_dim*2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# Adam(learning_rate=3e-4)
model.compile(
    optimizer=Adam(learning_rate=3e-4), loss="categorical_crossentropy", metrics=["categorical_accuracy"], 
)

(None, 512)


In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, 179)]  0           []                               
                                                                                                  
 bidirectional_1 (Bidirectional  [(None, 512),       892928      ['input_5[0][0]']                
 )                               (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

In [ ]:
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
training_limit = num_samples-num_test

In [ ]:
history = model.fit(
    [encoder_input_data[:training_limit], decoder_input_data[:training_limit]],
    decoder_target_data[:training_limit],
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[model_checkpoint_callback]
)

model.save(model_id)

Epoch 1/10
25/25 [==============================] - ETA: 0s - loss: 2.9151 - categorical_accuracy: 0.7678

25/25 [==============================] - 43s 2s/step - loss: 2.9151 - categorical_accuracy: 0.7678 - val_loss: 0.9463 - val_categorical_accuracy: 0.8509
Epoch 2/10
25/25 [==============================] - ETA: 0s - loss: 0.8338 - categorical_accuracy: 0.8495

25/25 [==============================] - 38s 2s/step - loss: 0.8338 - categorical_accuracy: 0.8495 - val_loss: 0.7601 - val_categorical_accuracy: 0.8509
Epoch 3/10
25/25 [==============================] - ETA: 0s - loss: 0.7422 - categorical_accuracy: 0.8495

25/25 [==============================] - 36s 1s/step - loss: 0.7422 - categorical_accuracy: 0.8495 - val_loss: 0.6958 - val_categorical_accuracy: 0.8509
Epoch 4/10
25/25 [==============================] - ETA: 0s - loss: 0.6959 - categorical_accuracy: 0.8503

25/25 [==============================] - 36s 1s/step - loss: 0.6959 - categorical_accuracy: 0.8503 - val_loss: 0.6707 - val_categorical_accuracy: 0.8509
Epoch 5/10
25/25 [==============================] - ETA: 0s - loss: 0.6740 - categorical_accuracy: 0.8500

...

25/25 [==============================] - 35s 1s/step - loss: 0.6575 - categorical_accuracy: 0.8505 - val_loss: 0.6420 - val_categorical_accuracy: 0.8526
Epoch 7/10
25/25 [==============================] - ETA: 0s - loss: 0.6424 - categorical_accuracy: 0.8512

25/25 [==============================] - 33s 1s/step - loss: 0.6424 - categorical_accuracy: 0.8512 - val_loss: 0.6342 - val_categorical_accuracy: 0.8556
Epoch 8/10
25/25 [==============================] - ETA: 0s - loss: 0.6323 - categorical_accuracy: 0.8552

25/25 [==============================] - 37s 2s/step - loss: 0.6323 - categorical_accuracy: 0.8552 - val_loss: 0.6267 - val_categorical_accuracy: 0.8552
Epoch 9/10
25/25 [==============================] - ETA: 0s - loss: 0.6274 - categorical_accuracy: 0.8556

25/25 [==============================] - 36s 1s/step - loss: 0.6274 - categorical_accuracy: 0.8556 - val_loss: 0.6225 - val_categorical_accuracy: 0.8535
Epoch 10/10
25/25 [==============================] - ETA: 0s - loss: 0.6059 - categorical_accuracy: 0.8580

25/25 [==============================] - 36s 1s/step - loss: 0.6059 - categorical_accuracy: 0.8580 - val_loss: 0.6006 - val_categorical_accuracy: 0.8571


In [ ]:
model = keras.models.load_model(model_id)

encoder_inputs = model.input[0]  # input_1
encoder_outputs, forward_h, forward_c, backward_h, backward_c = model.layers[1].output  # lstm_1
state_h_enc = Concatenate()([forward_h, backward_h])
state_c_enc = Concatenate()([forward_c, backward_c])
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim*2,))
decoder_state_input_c = keras.Input(shape=(latent_dim*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[5]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["<S>"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]

        if sampled_char == "<E>" or len(decoded_sentence) > max_decoder_seq_length:
          stop_condition = True
        else:
          decoded_sentence += sampled_char
          decoded_sentence += " "

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

# Test dataset

In [ ]:
unseen_file = open("unseen_data.txt", "w")
unseen_data_candidate = []
i = 0

for seq_index in range(num_samples-num_test,num_samples):
    i+=1
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    unseen_file.write("-\n")
    input_sentence = input_texts[seq_index]

    unseen_data_candidate.append(decoded_sentence)

    input_sentence = input_sentence.replace(" ","")
    input_sentence = input_sentence.replace("$"," ")

    unseen_file.write(input_sentence+"\n")
    unseen_file.write(decoded_sentence+"\n")

df_unseen_candidate = pd.DataFrame(unseen_data_candidate)
df_unseen_candidate.to_csv (r'unseen_candidate.csv', index = False, header=False)
unseen_file.close()

1/1 [==============================] - 0s 26ms/step


In [ ]:
unseen_file = open("unseen_data.txt", "w")
unseen_data_target = []
unseen_data_candidate = []
i = 0
for seq_index in range(0 ,num_samples):
    i+=1
    #print("unseen data ",i)
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    # print(input_seq.shape)
    decoded_sentence = decode_sequence(input_seq)
    unseen_file.write("-\n")
    input_sentence = input_texts[seq_index]
    target_sentence = target_texts[seq_index]
    target_sentence = target_sentence.replace("<S>", "")
    target_sentence = target_sentence.replace("<E>", "")

    unseen_data_target.append(target_sentence)
    unseen_data_candidate.append(decoded_sentence)

    input_sentence = input_sentence.replace(" ","")
    input_sentence = input_sentence.replace("$"," ")

    unseen_file.write(input_sentence+"\n")
    unseen_file.write(target_sentence + "\n")
    unseen_file.write(decoded_sentence+"\n")

df_unseen_target = pd.DataFrame(unseen_data_target)
df_unseen_candidate = pd.DataFrame(unseen_data_candidate)
df_unseen_target.to_csv (r'unseen_target.csv', index = False, header=False)
df_unseen_candidate.to_csv (r'unseen_candidate.csv', index = False, header=False)
unseen_file.close()

In [ ]:
seen_file = open("seen_data.txt", "w")
seen_data_target = []
seen_data_candidate = []
i=0
for seq_index in range(min(2000,num_samples - test_count)):
    i+=1
    #print("seen data ",i)
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    # print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    input_sentence = input_texts[seq_index]
    target_sentence = target_texts[seq_index]
    target_sentence = target_sentence.replace("<S>", "")
    target_sentence = target_sentence.replace("<E>", "")

    seen_data_target.append(target_sentence)
    seen_data_candidate.append(decoded_sentence)

    input_sentence = input_sentence.replace(" ","")
    input_sentence = input_sentence.replace("$"," ")

    seen_file.write("-\n")
    seen_file.write(input_sentence + "\n")
    seen_file.write(target_sentence + "\n")
    seen_file.write(decoded_sentence + "\n")

df_seen_target = pd.DataFrame(seen_data_target)
df_seen_candidate = pd.DataFrame(seen_data_candidate)
df_seen_target.to_csv (r'seen_target.csv', index = False, header=False)
df_seen_candidate.to_csv (r'seen_candidate.csv', index = False, header=False)
seen_file.close()

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
score = corpus_bleu(unseen_data_target, unseen_data_candidate)
print(score)